# This Notebook contains all the methods used for prediction NSCLC

contact: 

Saransh Gupta: saransh.official.iitkgp@gmail.com

Website: http://saranshqm.github.io/

Merge files to : https://github.com/saranshqm/NIBIOHN-NSCLC

Importing all required Libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from scipy import stats
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import joblib
import warnings
warnings.filterwarnings("ignore")

Importing the data-sets

In [2]:
train = pd.read_csv('file:///D:/remote%20projects/NIBIOHN/NSCLC/training%20files/non_oversampled_data/original_data_with_412_genes.csv')
train = train.drop(['Unnamed: 0'],axis=1)


try:
    train['SEPT4']

except:
    train.rename(columns={'Sep-04': 'SEPT4'}, inplace=True)
else:
    print('SEPT4 was present already')
    
    
    
test = pd.read_csv('file:///D:/remote%20projects/NIBIOHN/NSCLC/test%20files/test_after_features_selection.csv')
test = test.drop(['Unnamed: 0'],axis=1)

try:
    test['SEPT4']

except:
    test.rename(columns={'Sep-04': 'SEPT4'}, inplace=True)
else:
    print('SEPT4 was present already')

SEPT4 was present already
SEPT4 was present already


In [3]:
a = train.columns

# for i in a:
#     print(i)
import random
random.seed(2)
np.random.seed()

Outlier removal(using Z-test) ----> oversampling using SMOTE

In [4]:
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(train))
threshold = 3
original_train = train
train = train[(z < 3).all(axis=1)]
train.columns
y = train['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x = train[train.columns[:-1]]
x = np.asarray(x)
y = np.asarray(y)
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2)
# sm = SMOTE(random_state = 2) 
# x, y = sm.fit_sample(x, y.ravel())

Using TensorFlow backend.


In [5]:
x.shape

(507, 412)

# Weak Classifiers

1. Logistic Regression
2. Decision Trees Classifiers
3. Support Vector Machines
4. Deep Neural Networks (sequential modelling)

Logistic Regression  

params: C=0.05827593122724494,penalty='l1', solver='liblinear',
                                      tol=0.01, max_iter=10,
                                      warm_start=True,
                                      intercept_scaling=10,random_state = 2

In [6]:
print('Training and checking cross-validation scores\n')

clf = linear_model.LogisticRegression(C=0.05827593122724494,penalty='l1', solver='liblinear',
                                      tol=0.01, max_iter=10,
                                      warm_start=True,
                                      intercept_scaling=10,random_state = 2)
skf = StratifiedKFold(n_splits=3, random_state=42)
fold = 1
for train_index, val_index in skf.split(x,y): 
     
#     x, y = sm.fit_sample(x, y.ravel())
    #print("Train:", train_index, "Validation:", val_index) 
    X_train, X_test = x[train_index], x[val_index] 
    Y_train, Y_test = y[train_index], y[val_index]
    
    X_train, Y_train = sm.fit_sample(X_train, Y_train.ravel())
#     X_train, Y_train = sm.fit_sample(x, y.ravel())
    
#     x, y = sm.fit_sample(x, y.ravel())
    
    model = clf.fit(X_train,Y_train)
    a_train = model.predict(X_train)
    a_test = model.predict(X_test)
    print('K-FOLD #',fold)
    fold+=1
    print('Train AUC-ROC Score: ',roc_auc_score(Y_train,a_train),' Val AUC-ROC score: ',roc_auc_score(Y_test,a_test),'\n')
    print('validation kappa:-',cohen_kappa_score(Y_test,a_test))
print('Here we go with the confision Matrix\n')
print(confusion_matrix(Y_test,a_test))
print(classification_report(Y_test,a_test))


print('\n')
print('........................Testing the Model..............................\n')



y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x_testt = test[test.columns[:-1]]
y_testt_pred = model.predict(x_testt)

a_test = []
for i in y_testt_pred:
    if i>0.5:
        a_test.append(1)
    else:
        a_test.append(0)

print('0:- NSCLC Detected', '\t& \t1:- Normal\n')

print('Here we go with the confision Matrix\n')
print(confusion_matrix(y_testt,a_test))
print(classification_report(y_testt,a_test))

print('test kappa: \t',cohen_kappa_score(y_testt, a_test))
print('test roc score: \t',roc_auc_score(y_testt,a_test))

Training and checking cross-validation scores

K-FOLD # 1
Train AUC-ROC Score:  0.9672131147540983  Val AUC-ROC score:  0.9606731775374956 

validation kappa:- 0.8865581473401577
K-FOLD # 2
Train AUC-ROC Score:  0.9774590163934427  Val AUC-ROC score:  0.9393965817928148 

validation kappa:- 0.8563902107409925
K-FOLD # 3
Train AUC-ROC Score:  0.9692622950819673  Val AUC-ROC score:  0.9565748168817578 

validation kappa:- 0.8731760193446176
Here we go with the confision Matrix

[[114   8]
 [  1  46]]
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       122
           1       0.85      0.98      0.91        47

    accuracy                           0.95       169
   macro avg       0.92      0.96      0.94       169
weighted avg       0.95      0.95      0.95       169



........................Testing the Model..............................

0:- NSCLC Detected 	& 	1:- Normal

Here we go with the confision Matrix

[[191   8]
 [  0  19]

In [7]:
# boruta_feature_selector.support_weak_ 



import pickle 
  
# # Save the trained model as a pickle string. 
# saved_model = pickle.dumps(boruta_feature_selector) 


# from sklearn.externals import joblib 
  
# # Save the model as a pickle in a file 
joblib.dump(model, 'logistic_regression_KFOLD_OVERSAMPLE.pkl') 
  
# # Load the model from the file 
# # knn_from_joblib = joblib.load('filename.pkl')  
  
# # Use the loaded model to make predictions 
# # knn_from_joblib.predict(X_test)

['logistic_regression_KFOLD_OVERSAMPLE.pkl']

Decision Trees Classifiers (DTC)

params: class_weight=None, criterion='gini', max_depth=int(10),
                       max_features=int(10.4),
                       min_samples_leaf=int(10), min_samples_split=2,
                       random_state=42, splitter='best'

In [8]:
clf = tree.DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=int(10),
                       max_features=int(10.4),
                       min_samples_leaf=int(10), min_samples_split=2,
                       random_state=42, splitter='best')

skf = StratifiedKFold(n_splits=3, random_state=42)
# X is the feature set and y is the target
fold = 1
for train_index, val_index in skf.split(x,y): 
#     x, y = sm.fit_sample(x, y.ravel())
    #print("Train:", train_index, "Validation:", val_index) 
    X_train, X_test = x[train_index], x[val_index] 
    Y_train, Y_test = y[train_index], y[val_index]
    
    X_train, Y_train = sm.fit_sample(X_train, Y_train.ravel())
    
    model = clf.fit(X_train,Y_train)
    a_train = model.predict(X_train)
    a_test = model.predict(X_test)
    print('K-FOLD #',fold)
    fold+=1
    print('Train AUC-ROC Score: ',roc_auc_score(Y_train,a_train),' Val AUC-ROC score: ',roc_auc_score(Y_test,a_test),'\n')

    print('val kappa:-',cohen_kappa_score(Y_test,a_test))

print(confusion_matrix(Y_test,a_test))
print(classification_report(Y_test,a_test))



print('\n')
print('........................Testing the Model..............................\n')



y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x_testt = test[test.columns[:-1]]
y_testt_pred = model.predict(x_testt)

a_test = []
for i in y_testt_pred:
    if i>0.5:
        a_test.append(1)
    else:
        a_test.append(0)

print('0:- NSCLC Detected', '\t& \t1:- Normal\n')

print('Here we go with the confision Matrix\n')
print(confusion_matrix(y_testt,a_test))
print(classification_report(y_testt,a_test))

print('test kappa: \t',cohen_kappa_score(y_testt, a_test))
print('test roc score: \t',roc_auc_score(y_testt, a_test))


K-FOLD # 1
Train AUC-ROC Score:  0.9590163934426229  Val AUC-ROC score:  0.8968433903034531 

val kappa:- 0.7936867806069061
K-FOLD # 2
Train AUC-ROC Score:  0.9651639344262294  Val AUC-ROC score:  0.8927450296477154 

val kappa:- 0.7803863813566664
K-FOLD # 3
Train AUC-ROC Score:  0.9426229508196722  Val AUC-ROC score:  0.8755667945587723 

val kappa:- 0.7610885315426754
[[115   7]
 [  9  38]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       122
           1       0.84      0.81      0.83        47

    accuracy                           0.91       169
   macro avg       0.89      0.88      0.88       169
weighted avg       0.90      0.91      0.90       169



........................Testing the Model..............................

0:- NSCLC Detected 	& 	1:- Normal

Here we go with the confision Matrix

[[193   6]
 [  0  19]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      

In [9]:
import pickle 
  
# # Save the trained model as a pickle string. 
# saved_model = pickle.dumps(boruta_feature_selector) 


# from sklearn.externals import joblib 
  
# # Save the model as a pickle in a file 
joblib.dump(model, 'decision_trees_KFOLD_OVERSAMPLED.pkl') 

['decision_trees_KFOLD_OVERSAMPLED.pkl']

Support Vector Machines (SVM)

params: kernel='linear',gamma = 82, C = 1.4555047314930398

In [10]:
clf =  SVC(probability = True, kernel='linear',gamma = 82, C = 1.4555047314930398)
k_fold = 3
skf = StratifiedKFold(n_splits=k_fold, random_state=42)
fold = 1
for train_index, val_index in skf.split(x,y): 
#     x, y = sm.fit_sample(x, y.ravel())
    #print("Train:", train_index, "Validation:", val_index) 
    X_train, X_test = x[train_index], x[val_index] 
    Y_train, Y_test = y[train_index], y[val_index]
    
    X_train, Y_train = sm.fit_sample(X_train, Y_train.ravel())
    
    model = clf.fit(X_train,Y_train)
    a_train = model.predict(X_train)
    a_test = model.predict(X_test)
    print('K-FOLD #',fold)
    fold+=1
    print('Train AUC-ROC Score: ',roc_auc_score(Y_train,a_train),' Val AUC-ROC score: ',roc_auc_score(Y_test,a_test),'\n')
    print('validation kappa:-',cohen_kappa_score(Y_test,a_test))
    

print(confusion_matrix(Y_test,a_test))
print(classification_report(Y_test,a_test))
print('\n')
print('........................Testing the Model..............................\n')



y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x_testt = test[test.columns[:-1]]
y_testt_pred = model.predict(x_testt)

a_test = []
for i in y_testt_pred:
    if i>0.5:
        a_test.append(1)
    else:
        a_test.append(0)

print('0:- NSCLC Detected', '\t& \t1:- Normal\n')

print('Here we go with the confision Matrix\n')
print(confusion_matrix(y_testt,a_test))
print(classification_report(y_testt,a_test))

print('test kappa: \t',cohen_kappa_score(y_testt, a_test))
print('test roc score: \t',roc_auc_score(y_testt, a_test))

K-FOLD # 1
Train AUC-ROC Score:  1.0  Val AUC-ROC score:  0.9369550052319496 

validation kappa:- 0.8682318288139999
K-FOLD # 2
Train AUC-ROC Score:  1.0  Val AUC-ROC score:  0.918119986048134 

validation kappa:- 0.8254432776725771
K-FOLD # 3
Train AUC-ROC Score:  1.0  Val AUC-ROC score:  0.9254447157307291 

validation kappa:- 0.7937850809403726
[[109  13]
 [  2  45]]
              precision    recall  f1-score   support

           0       0.98      0.89      0.94       122
           1       0.78      0.96      0.86        47

    accuracy                           0.91       169
   macro avg       0.88      0.93      0.90       169
weighted avg       0.92      0.91      0.91       169



........................Testing the Model..............................

0:- NSCLC Detected 	& 	1:- Normal

Here we go with the confision Matrix

[[162  37]
 [  0  19]]
              precision    recall  f1-score   support

           0       1.00      0.81      0.90       199
           1       0

In [11]:
joblib.dump(model, 'svm_KFOLD_OVERSAMPLED.pkl') 

['svm_KFOLD_OVERSAMPLED.pkl']

Deep Neural Networks (sequential modelling)

# Strong Classifiers

1. CATBOOST (GPU Powered)
2. Random Forest
3. XGBOOST
4. LGBM

1. CATBOOST

params: loss_function= 'Logloss',iterations = 300,
    learning_rate=.01,
    max_depth=10, eval_metric = 'AUC',leaf_estimation_iterations = 10,task_type="GPU",
                           devices='0-6'

In [12]:
kfold = 3
np.random.seed(2)
skf = StratifiedKFold(n_splits=kfold, random_state=42)

clf = CatBoostClassifier(loss_function= 'Logloss',iterations = 300,
    learning_rate=.01,
    max_depth=10, eval_metric = 'AUC',leaf_estimation_iterations = 10,task_type="GPU",
                           devices='0-6')
fold = 1
for train_index, val_index in skf.split(x,y): 
#     x, y = sm.fit_sample(x, y.ravel())
    X_train, X_test = x[train_index], x[val_index] 
    Y_train, Y_test = y[train_index], y[val_index]
    
    X_train, Y_train = sm.fit_sample(X_train, Y_train.ravel())
    
    mdl = clf.fit(X_train,Y_train,verbose=False)
    a_train = mdl.predict(X_train)
    a_test = mdl.predict(X_test)
    print('K-FOLD #',fold)
    fold+=1
    print('Train AUC-ROC Score: ',roc_auc_score(Y_train,a_train),' Val AUC-ROC score: ',roc_auc_score(Y_test,a_test),'\n')
    print('validation kappa:-',cohen_kappa_score(Y_test,a_test))

K-FOLD # 1
Train AUC-ROC Score:  1.0  Val AUC-ROC score:  0.9475933031042901 

validation kappa:- 0.883628851781718
K-FOLD # 2
Train AUC-ROC Score:  1.0  Val AUC-ROC score:  0.9623299616323684 

validation kappa:- 0.9127216388362885
K-FOLD # 3
Train AUC-ROC Score:  1.0  Val AUC-ROC score:  0.9516916637600279 

validation kappa:- 0.897513644633111


In [13]:
joblib.dump(mdl, 'catboost_KFOLD_OVERSAMPLING.pkl')

['catboost_KFOLD_OVERSAMPLING.pkl']

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test,a_test))
print(classification_report(Y_test,a_test))

[[118   4]
 [  3  44]]
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       122
           1       0.92      0.94      0.93        47

    accuracy                           0.96       169
   macro avg       0.95      0.95      0.95       169
weighted avg       0.96      0.96      0.96       169



In [15]:
y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x_testt = test[test.columns[:-1]]
y_test_test = mdl.predict(x_testt)
print('0:- NSCLC Detected', '\t& \t1:- Normal\n')
print(confusion_matrix(y_testt,y_test_test))
print(classification_report(y_testt,y_test_test))

print('test kappa:-',cohen_kappa_score(y_testt, y_test_test))
print('test roc score',roc_auc_score(y_testt, y_test_test))

0:- NSCLC Detected 	& 	1:- Normal

[[199   0]
 [  9  10]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       199
           1       1.00      0.53      0.69        19

    accuracy                           0.96       218
   macro avg       0.98      0.76      0.83       218
weighted avg       0.96      0.96      0.95       218

test kappa:- 0.6698081454055873
test roc score 0.763157894736842


2. Random Forest

params:n_estimators=int(10.27),bootstrap = True, max_depth = 3, max_features = 8, max_leaf_nodes = 17, n_jobs = 9, random_state = 42

In [16]:
clf = RandomForestClassifier(n_estimators=int(10.27),bootstrap = True, max_depth = 3, max_features = 8, max_leaf_nodes = 17, n_jobs = 9, random_state = 42)
skf = StratifiedKFold(n_splits=3, random_state=42)
# X is the feature set and y is the target
fold = 1
for train_index, val_index in skf.split(x,y): 
#     x, y = sm.fit_sample(x, y.ravel())
    #print("Train:", train_index, "Validation:", val_index) 
    X_train, X_test = x[train_index], x[val_index] 
    Y_train, Y_test = y[train_index], y[val_index]
    
    X_train, Y_train = sm.fit_sample(X_train, Y_train.ravel())
#     X_test, Y_test = sm.fit_sample(X_test, Y_test.ravel())
    
    
    model = clf.fit(X_train,Y_train)
    a_train = model.predict(X_train)
    a_test = model.predict(X_test)
    print('K-FOLD #',fold)
    fold+=1
    print('Train AUC-ROC Score: ',roc_auc_score(Y_train,a_train),' Val AUC-ROC score: ',roc_auc_score(Y_test,a_test),'\n')
    print('validation kappa:-',cohen_kappa_score(Y_test,a_test))

print(confusion_matrix(Y_test,a_test))
print(classification_report(Y_test,a_test))


print('\n')
print('........................Testing the Model..............................\n')



y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x_testt = test[test.columns[:-1]]
y_testt_pred = model.predict(x_testt)

a_test = []
for i in y_testt_pred:
    if i>0.5:
        a_test.append(1)
    else:
        a_test.append(0)

print('0:- NSCLC Detected', '\t& \t1:- Normal\n')

print('Here we go with the confision Matrix\n')
print(confusion_matrix(y_testt,a_test))
print(classification_report(y_testt,a_test))

print('test kappa: \t',cohen_kappa_score(y_testt, a_test))
print('test roc score: \t',roc_auc_score(y_testt, a_test))

K-FOLD # 1
Train AUC-ROC Score:  0.9774590163934426  Val AUC-ROC score:  0.8985001743983257 

validation kappa:- 0.8184097421203438
K-FOLD # 2
Train AUC-ROC Score:  0.9774590163934427  Val AUC-ROC score:  0.9434949424485525 

validation kappa:- 0.8699221756606517
K-FOLD # 3
Train AUC-ROC Score:  0.9692622950819672  Val AUC-ROC score:  0.9516916637600279 

validation kappa:- 0.897513644633111
[[118   4]
 [  3  44]]
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       122
           1       0.92      0.94      0.93        47

    accuracy                           0.96       169
   macro avg       0.95      0.95      0.95       169
weighted avg       0.96      0.96      0.96       169



........................Testing the Model..............................

0:- NSCLC Detected 	& 	1:- Normal

Here we go with the confision Matrix

[[199   0]
 [ 14   5]]
              precision    recall  f1-score   support

           0       0.93      

In [17]:
# x.shape

In [18]:
joblib.dump(model, 'random_forest_KFOLD_OVERSAMPLING.pkl')

['random_forest_KFOLD_OVERSAMPLING.pkl']

3. XGBOOST

params: {
    'min_child_weight': 10.0,
    'lambda': 0.3397604475560514,
    'objective': 'binary:logistic',
    
    'max_depth': 20,
    'alpha': 5.7154910613882356e-05,
    'max_delta_step': 0.1,
    'eta': 0.1673343679061888,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.1673343679061888,
    'gamma': 1.019025732933819e-06,
    'grow_policy': 'depthwise',
    'eval_metric' : 'auc',
    'silent': 1,
    'num_boost_round' : 700,
    
    }

In [19]:
kfold = 4
skf = StratifiedKFold(n_splits=kfold, random_state=42)

params = {
    'min_child_weight': 10.0,
    'lambda': 0.3397604475560514,
    'objective': 'binary:logistic',
    
    'max_depth': 20,
    'alpha': 5.7154910613882356e-05,
    'max_delta_step': 0.1,
    'eta': 0.1673343679061888,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.1673343679061888,
    'gamma': 1.019025732933819e-06,
    'grow_policy': 'depthwise',
    'eval_metric' : 'auc',
    'silent': 1,
    'num_boost_round' : 700,
    
    }


import xgboost as xgb
from sklearn.metrics import roc_auc_score
for i, (train_index, test_index) in enumerate(skf.split(x, y)):
#     x, y = sm.fit_sample(x, y.ravel())
    print('[Fold %d/%d]' % (i + 1, kfold))
    X_train, X_valid = x[train_index], x[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    
    X_train, y_train = sm.fit_sample(X_train, y_train.ravel())
    
    # Convert our data into XGBoost format
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    mdl = xgb.train(params, d_train, 30000, watchlist, early_stopping_rounds=1000, maximize=True,verbose_eval = 1000)
    a_valid = mdl.predict(d_valid)
    
    a = []
    for i in a_valid:
        if i>0.6:
            a.append(1)
        else:
            a.append(0)
    
    print('val kappa:-',cohen_kappa_score(y_valid, a))
    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))

[Fold 1/4]
[12:30:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { num_boost_round, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.94343	valid-auc:0.81025
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 1000 rounds.
[1000]	train-auc:0.99980	valid-auc:0.96491
Stopping. Best iteration:
[175]	train-auc:0.99960	valid-auc:0.96646

val kappa:- 0.8027950310559007
[Fold 1/4 Prediciton:]
[Fold 2/4]
[12:30:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { num_boost_round, silent } might not be used.

  This may not be accurate due to some parameters are only used 

In [20]:
joblib.dump(mdl, 'xgboost_KFOLD_OVERSAMPLING.pkl')

['xgboost_KFOLD_OVERSAMPLING.pkl']

testing on independent test data set using XGBOOST

In [21]:
y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x_testt = test[test.columns[:-1]]
x_testt = xgb.DMatrix(np.array(x_testt),y_testt)
y_testt_pred = mdl.predict(x_testt)

a_test = []
for i in y_testt_pred:
    if i>0.6:
        a_test.append(1)
    else:
        a_test.append(0)
print('0:- NSCLC Detected', '\t& \t1:- Normal\n')
print(confusion_matrix(y_testt,a_test))
print(classification_report(y_testt,a_test))
print('test kappa:-',cohen_kappa_score(y_testt, a_test))
print('test roc score',roc_auc_score(y_testt, a_test))

0:- NSCLC Detected 	& 	1:- Normal

[[199   0]
 [  0  19]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       199
           1       1.00      1.00      1.00        19

    accuracy                           1.00       218
   macro avg       1.00      1.00      1.00       218
weighted avg       1.00      1.00      1.00       218

test kappa:- 1.0
test roc score 1.0


4. LGBM:

params: iterations = 1000,
    subsample=.8,
    learning_rate=.1,
    max_depth=10, max_delta_step = 0.1, alpha = 5.71, min_child_weight = 10, eta = 0.167, cosample_bytree = 0.4,
    num_boost_round = 700

In [22]:
kfold = 3
skf = StratifiedKFold(n_splits=kfold, random_state=42)

clf = lgb.LGBMClassifier(iterations = 1000,
    subsample=.8,
    learning_rate=.1,
    max_depth=10, max_delta_step = 0.1, alpha = 5.71, min_child_weight = 10, eta = 0.167, cosample_bytree = 0.4,
    num_boost_round = 700)
fold = 1
for train_index, val_index in skf.split(x,y):
#     x, y = sm.fit_sample(x, y.ravel())
    #print("Train:", train_index, "Validation:", val_index) 
    X_train, X_test = x[train_index], x[val_index] 
    Y_train, Y_test = y[train_index], y[val_index]
    
    X_train, Y_train = sm.fit_sample(X_train, Y_train.ravel())
    
    mdl = clf.fit(X_train,Y_train)
    a_train = mdl.predict(X_train)
    a_test = mdl.predict(X_test)
    print('K-FOLD #',fold)
    fold+=1
    print('Train AUC-ROC Score: ',roc_auc_score(Y_train,a_train),' Val AUC-ROC score: ',roc_auc_score(Y_test,a_test),'\n')
    print('validation kappa:-',cohen_kappa_score(Y_test,a_test))

[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] Unknown parameter: cosample_bytree
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.167 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] num_iterations is set=700, num_boost_round=700 will be ignored. Current value: num_iterations=700
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
K-FOLD # 1
Train AUC-ROC Score:  0.9959016393442625  Val AUC-ROC score:  0.9197767701430066 

validation kappa:- 0.850680332214172
[LightGBM] [Warning] Unknown parameter: iterations
[LightGBM] [Warning] Unknown parameter: cosample_bytree
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.167 will be ignored. Current value: learning_rate=0.1
[LightGBM] [Warning] num_iterations is set=700, num_boost_round=700 will be ignored. Current value: num_iterations=700
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set 

In [23]:
joblib.dump(mdl, 'LGBM_KFOLDOVERSAMPLING.pkl')

['LGBM_KFOLDOVERSAMPLING.pkl']

Testing LGBM on independent test data-set

In [24]:
y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x_testt = test[test.columns[:-1]]
y_testt_pred = mdl.predict(x_testt)

a_test = []
for i in y_testt_pred:
    if i>0.6:
        a_test.append(1)
    else:
        a_test.append(0)
print('0:- NSCLC Detected', '\t& \t1:- Normal\n')
print(confusion_matrix(y_testt,a_test))
print(classification_report(y_testt,a_test))
print('test kappa:-',cohen_kappa_score(y_testt, a_test))
print('test roc score',roc_auc_score(y_testt, a_test))

0:- NSCLC Detected 	& 	1:- Normal

[[197   2]
 [  0  19]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       199
           1       0.90      1.00      0.95        19

    accuracy                           0.99       218
   macro avg       0.95      0.99      0.97       218
weighted avg       0.99      0.99      0.99       218

test kappa:- 0.944963393082555
test roc score 0.9949748743718593
